<h2>Taller 3 - Extra Bonus - Descripción Denavit-Hartenberg y cinemática directa</h2>
<br>Autor: Claudio Morales D.
<br>Centro de Automatización y Robótica Inacap
<br>Santiago, Chile, enero 2023


Utilizamos la librería sympy para calcular la matriz de transformación de coordenadas homogéneas a partir de la descripción Devaniv-Hartenberg para un robot SCARA. A partir de esta matriz, se extraen las ecuaciones de cinemática directa para este robot.

<h3>0. Verificamos que existe comunicación con CoppeliaSim</h3>
Varios ejemplos del uso de la zeroMQ remote API pueden consultarse aquí: <a>https://github.com/CoppeliaRobotics/zmqRemoteApi/tree/master/clients/python</a>
Una descripción de las funciones incluidas en la API se encuentra aquí: <a>https://www.coppeliarobotics.com/helpFiles/en/apiFunctions.htm</a>

In [1]:
# Cargamos la librería, creamos un cliente y obtenemos acceso a sim
import zmqRemoteApi
client = zmqRemoteApi.RemoteAPIClient()
sim = client.getObject('sim')

# Obtenemos los manejadores para las articulaciones y el actuador final
j1 = sim.getObject('/base/joint1')
j2 = sim.getObject('/base/joint2')
j3 = sim.getObject('/base/joint3')
conn = sim.getObject('/base/connection')

# y movemos a una posicion deseada
q1_val = 80 * 3.1416/180
q2_val = 0.0
q3_val = 60 * 3.1416/180

sim.setJointTargetPosition(j1, q1_val)
sim.setJointTargetPosition(j2, q2_val)
sim.setJointTargetPosition(j3, q3_val)

1

In [ ]:
# y retornamos a la posicion inicial
sim.setJointTargetPosition(j1, 0)
sim.setJointTargetPosition(j2, 0)
sim.setJointTargetPosition(j3, 0)

<h3>1. Obtenemos las matrices de transformación a partir de la descripción D-H </h3>


In [ ]:
# Comenzaremos el trabajo desde la descripción de Denavit-Hartenberg para el robot
#  link |     theta     |      d      |      a      |    alpha
# --------------------------------------------------------------
#   1           q1           0.13           0            0
#   2            0             q2         0.2            0
#   3           q3          -0.052        0.2           180

In [2]:
# importamos las librerías requeridas
import sympy

In [3]:
# Definimos una función para construir las matrices de transformación
# en forma simbóĺica a partir de los parámetros D-H

def MatrixfromDH(theta, d, a, alpha):
    # theta y alpha en radianes
    # d y a en metros
    Rz = sympy.Matrix([[sympy.cos(theta), -sympy.sin(theta), 0, 0],
                   [sympy.sin(theta), sympy.cos(theta), 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    tz = sympy.Matrix([[1, 0, 0, 0],
                   [0, 1, 0, 0],
                   [0, 0, 1, d],
                   [0, 0, 0, 1]])
    ta = sympy.Matrix([[1, 0, 0, a],
                   [0, 1, 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    Rx = sympy.Matrix([[1, 0, 0, 0],
                   [0, sympy.cos(alpha), -sympy.sin(alpha), 0],
                   [0, sympy.sin(alpha), sympy.cos(alpha), 0],
                   [0, 0, 0, 1]])
    return Rz*tz*ta*Rx

In [8]:
# Obtenemos la matriz de transformación para el primer eslabon
q1, q2, q3 = sympy.symbols(['q1', 'q2', 'q3'])
j1_T_j2 = MatrixfromDH(q1, 0.13, 0, 0)
j2_T_j3 = MatrixfromDH(0, q2, 0.2, 0)
j3_T_conn = MatrixfromDH(q3, -0.052, 0.2, sympy.pi)
j1_T_conn = j1_T_j2 * j2_T_j3 * j3_T_conn
j1_T_conn

Matrix([
[-sin(q1)*sin(q3) + cos(q1)*cos(q3), sin(q1)*cos(q3) + sin(q3)*cos(q1),  0, -0.2*sin(q1)*sin(q3) + 0.2*cos(q1)*cos(q3) + 0.2*cos(q1)],
[ sin(q1)*cos(q3) + sin(q3)*cos(q1), sin(q1)*sin(q3) - cos(q1)*cos(q3),  0,  0.2*sin(q1)*cos(q3) + 0.2*sin(q1) + 0.2*sin(q3)*cos(q1)],
[                                 0,                                 0, -1,                                               q2 + 0.078],
[                                 0,                                 0,  0,                                                        1]])

<h3>2. Evaluando la cinemática directa </h3>


In [10]:
# creamos las magnitudes que queremos evaluar
q1_val = 45 * 3.1416/180
q2_val = 0.1
q3_val = 0

# movemos el robot a la posición
sim.setJointTargetPosition(j1, q1_val)
sim.setJointTargetPosition(j2, q2_val)
sim.setJointTargetPosition(j3, q3_val)

#obtenemos la posición del extremo
pos = sim.getObjectPosition(conn, sim.handle_world)
print('La posición del extremo es', pos)

#y calculamos por cinemática directa para comparar
transf_matrix = j1_T_conn.subs({q1:q1_val, q2:q2_val, q3:q3_val})
print('La matriz de transformación correspondiente es:')
transf_matrix

La posición del extremo es [0.28284382820129395, 0.2828052043914795, 0.17783156037330627]
La matriz de transformación correspondiente es:


Matrix([
[0.707105482511236,  0.707108079859474,  0, 0.282842193004495],
[0.707108079859474, -0.707105482511236,  0, 0.282843231943789],
[                0,                  0, -1,             0.178],
[                0,                  0,  0,                 1]])

<h3>Extra: Redondeando los elementos </h3>
Una ayuda desde: <a>https://stackoverflow.com/questions/48491577/printing-the-output-rounded-to-3-decimals-in-sympy</a>

In [ ]:
def round_expr(expr, num_digits):
    return expr.xreplace({n : round(n, num_digits) for n in expr.atoms(sympy.Number)})